In [ ]:
!pip install tabula-py
!pip install tabular
!pip install PyPDF2

In [ ]:
import nltk
import PyPDF2
from tabula import read_pdf 
from tabulate import tabulate
import os
import re

### PREPROCESADO Y LIMPIEZA

Dada la empresa a realizar el rating tendremos una carpeta con su nombre. Dentro de esta carpeta habra una coleccion de carpetas con los años de historico. En el momento actual tenemos un historico desde el año 2021 hasta 2017. En cada una de estas carpetas encontramos los informes para dicho año.

En este script se procede a realizar un preprocesado y limpieza de los informes. Estos informes estan en PDF y habra que converitrlos a txt. Se realizara una limpieza del texto.

Ademas se realiza la extracción del contenido de las tablas ya que en muchos casos contienen información relevante y el modelo ha sido entrenado solo con texto. Para ello se convierte la información de las tablas en frases.
Si en las tablas aparace la evolución historica solo extrae la información
del año que estamos buscando

#### FUNCIONES A UTILIZAR

In [ ]:
def años_con_datos(empresa):
  '''
  Dada una empresa devuelve el conjunto de carpetas anuales que tiene, devolviendo asi el historico.
  Args:
        empresa (str): nombre de la empresa de estudio

    Returns:
        lista(str): que contiene los años para los que la empresa tiene datos
  '''
  path_origen='/content/drive/MyDrive/pdfs-prueba/'+empresa+'/'
  contenido = os.listdir(path_origen)
  
  return contenido



In [ ]:
def archivos(empresa,año):
  '''
  Dada una empresa y un año devuelve el conjunto de docuementos que tiene
  Args:
        empresa (str): nombre de la empresa de estudio
        año (str): año de estudio

    Returns:
        lista(str): que contiene los nombres de los informes
  '''
  path_origen='/content/drive/MyDrive/pdfs-prueba/'+empresa+'/'+año+'/'
  contenido = os.listdir(path_origen)
  
  return contenido

In [ ]:
def limpieza_passages(texto):
  '''
  Dado un texto realiza una limpieza caracteres especiales, transformar a minusculas y eleiminar urls
  Args:
        texto (str): texto a limpiar

    Returns:
        text(str): texto limpio
  '''
  texto=texto.replace("\n"," ")
  texto=texto.replace("\t"," ")
  texto=texto.replace("\\r"," ")
  texto=texto.replace("\r"," ")
  texto=texto.replace(" - ","")
  texto=texto.replace("- ","")
  texto=texto.replace("-","")
  texto=texto.replace(" -","")
  texto=texto.replace(">"," ")
  texto=texto.lower()
  texto = re.sub(r'^https?:\/\/.*[\r\n]*', '', texto, flags=re.MULTILINE)
 
  return texto

In [ ]:
def pdf_to_txt(empresa, año, nombre):
  '''
  Dada un archivo pdf lo convierte a txt realizando la limpieza de su texto y lo deja en la misma ruta
  Args:
        empresa (str): nombre de la empresa de estudio
        año (str): año de estudio
        nombre (str): nombre del informe a limpiar

  '''

  input_path='/content/drive/MyDrive/pdfs-prueba/'+empresa+'/'+año+'/'+nombre+'.pdf'
  if os.path.exists('/content/drive/MyDrive/DATO-ingles/'+empresa+'_tabla/')==False:
    os.mkdir('/content/drive/MyDrive/DATO-ingles/'+empresa+'_tabla/')
  output_path = '/content/drive/MyDrive/DATO-ingles/'+empresa+'_tabla/'+año+'/'+nombre+'_limpio.txt'  
 
  if os.path.exists('/content/drive/MyDrive/DATO-ingles/'+empresa+'_tabla/'+año+'/')==False:
    os.mkdir('/content/drive/MyDrive/DATO-ingles/'+empresa+'_tabla/'+año+'/')      
  pdfFileObj = open(input_path, 'rb')  
  pdfReader = PyPDF2.PdfReader(pdfFileObj)  
  print('num_paginas: ',len(pdfReader.pages))
  for i in range(0, len(pdfReader.pages)):
    
    pageObj = pdfReader.pages[i]
    
      
    texto=pageObj.extract_text()
    texto_limpio=limpieza_passages(texto)    
    f = open(output_path, 'a')
    f.write(texto_limpio)
    f.close()

In [ ]:
def columnas_son_años(columnas):
  '''
  Dado una lista de columnas determina si son años 
  Args:
        columnas list(str): nombre de las columnas de una tabla
       
  Return:
      Bool: indica si son años o no
  '''
  años=['2019','2020','2021','2018','2017']
  años = set(años)
  columnas = set(columnas)
  son=False
  interseccion= set(columnas).intersection(años)
  interseccion = list(interseccion)

  
  if len(interseccion)>1:
    son=True
  return son 

In [ ]:
def tabla_to_lines(df,año_actual):
  '''
  Extrae la informacion de una tabla a un texto. Ademas si las columnas son años, solo extrae la informacion del año que estamos estudiando
  Args:
        df Dataframe: tabla en formato dataframe de la que queremos extraer la informacion.
        año_actual str: año del que hacemos el estudio
       
  Return:
      txt: con la informacion de la tabla
  '''
  columnas=list(df.columns)
  num_columnas=df.shape[1]
  num_filas=df.shape[0]
  if num_filas>0 and num_columnas>0:
    titulo=titulo=columnas[0]
    filas=list(df.iloc[0:,0])
    texto=''
    if columnas_son_años(columnas[1:])==False:
      for i in range(0,num_filas):
        
        for j in range(1,num_columnas):
          
          dato=df.iloc[i,j]
          if str(dato)!='Nan' and str(dato)!='na' and str(dato)!='NaN' and str(dato)!='nan':
            if str(columnas[j])[0:8]!='Unnamed:':
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[j])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(columnas[j])+'             '
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[j])+'             '
                else: 
                  texto=texto+' '+str(dato)+' '+str(columnas[j])+'             '
              
            else:
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+'             '
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+'             '
                else: 
                  texto=texto+' '+str(dato)+'                  '
    else:
      indice_actual=columnas.index(año_actual)
      for i in range(0,num_filas):
        
        
          
          dato=df.iloc[i,indice_actual]
          if str(dato)!='Nan' and str(dato)!='na' and str(dato)!='NaN' and str(dato)!='nan':
            
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':

                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[indice_actual])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(columnas[indice_actual])+'             '
                
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[indice_actual])+'             '
                else: 
                  texto=texto+' '+str(dato)+' '+str(columnas[indice_actual])+'                  '
             
    return limpieza_passages(texto)

In [ ]:
def incluir_tablas(empresa,año,nombre):
  '''
  Dado un documento extrae para cada pagina sus tablas en formato DataFrame y posteriormente llama a la funcion
  tabla_to_lines para convertirla en texto. 
  
  
  Args:
        empresa str: empresa de busqueda
        año str: año de busqueda
        nombre str: nombre del documento del que vamos a extraer las tablas
       
 
  '''
  input_path='/content/drive/MyDrive/pdfs-prueba/'+empresa+'/'+año+'/'+nombre+'.pdf'
  pdfFileObj = open(input_path, 'rb')  
  pdfReader = PyPDF2.PdfReader(pdfFileObj) 
  num_paginas=len(pdfReader.pages)
  for i in range(0,num_paginas): 
    try:
      df = read_pdf(input_path,pages=str(i)) 
      output_path = '/content/drive/MyDrive/DATO-ingles/iteracion_3/'+empresa+'/'+año+'/'+nombre+'_tabla.txt' 
      with open(output_path, 'a') as f:
        for j in range(0,len(df)):
            texto_tabla=tabla_to_lines(df[j],año)
            if texto_tabla is not None:
              f.write(texto_tabla)
    
    except Exception as inst:
      
      pass

### PROCESO A SEGUIR PARA CONSEGUIR LOS RESULTADOS DE LAS EMPRESAS DEL IBEX 35

Definimos la empresa de la que queremos hacer el estudio

---



In [ ]:
empresas_IBEX35=['acciona','acerinox','acs','am','aena','amadeus','bankinter','bbva','caixabank','cellnex','colonial',
                 'enagas','endesa','ferrovial','fluidra','grifols','iag','iberdrola','indra','inditex','mapfre','melia',
                 'merlin','naturgy','red electrica','pharmamar','repsol','rovi','sacyr','santander','sabadell',
                 'siemens','solaria','telefonica']

In [ ]:
for empresa in empresas_IBEX35:
#años de historico que la empresa tiene de datos
  años=años_con_datos(empresa)
  for i in años:
    #para cada año vemos que archivos tiene
    contenido=archivos(empresa,año=i)
    for ar in contenido:
      # se convierte cada archivo a txt y se realiza su limpieza
      nombre_ar=ar[:-4]
      
      pdf_to_txt(empresa, año=i, nombre=nombre_ar)
      

In [ ]:
for empresa in empresas_IBEX35:
  for i in años:
    #para cada año vemos que archivos tiene
    contenido=archivos(empresa,año=i)
    for ar in contenido:
      nombre_ar=ar[:-4]
      try:
        #para cada archivo se extrae la informacion de sus tablas
        incluir_tablas(empresa, año=i, nombre=nombre_ar)
      except:
        pass